In [3]:
import glob
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(verbose=False)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
from openai import OpenAI
client = OpenAI()

audio_file = open("../../data/소아남여_소아남여01_F_1592580488-0_7_수도권_실내_00002.wav", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text",  
)
print(transcription)

우리 삼촌은 베트남에 있어.



In [12]:
transcription

'우리 이모는 미국 살아.\n'